IMPORTAÇÃO DE BIBLIOTECAS E CRIAÇÃO DE VARIÁVEIS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import math

path_input = r"C:\Documentos\Rede Neural\input.xlsx"

ORGANIZAÇÃO DO INPUT E PRÉ PROCESSAMENTO

In [ ]:
df = pd.read_excel(path_input ) #criação do dataframe de input

X = df.drop(['Produção de óleo'], axis=1) #divisão do dataframe entre input e output
X = X.drop(['simulação'], axis=1)
X = X.drop(['Current oil in place'], axis=1)
X = X.drop(['Original oil in place'], axis=1)
num_parameters = len(X.columns) #contagem do número de variáveis
y = pd.DataFrame(df['Produção de óleo']).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) #divisão do dataframe em grupos de treinamento e teste

scaler = StandardScaler() #normalização dos dados
scaler.fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
num_of_samples = len(df.index) #definição do número de pontos

CRIAÇÃO DA REDE NEURAL

In [ ]:
model = Sequential() #definição do modelo da rede neural

model.add(Dense(num_parameters,input_dim=num_parameters, activation='relu')) #definição dos parâmetros da rede neural
model.add(Dense(int(math.sqrt((3*len(df.index))) + (2*(len(df.index)/3))), activation='relu'))
model.add(Dense(int((len(df.index)/3)), activation='relu'))
model.add(Dense(1, activation='linear'))

opt = keras.optimizers.Adam(learning_rate=0.002) #definição do optimizer
callback = keras.callbacks.EarlyStopping(monitor='mae', patience=50, restore_best_weights=True) #definição do critério de parada
model.compile(loss='mae', optimizer = opt ,metrics=['mse', 'mae']) #compilação do modelo

history = model.fit(X_train, y_train, epochs=10000, validation_data=(X_test,y_test), callbacks=[callback]) #treinamento e testagem do modelo